## Query for the weather data for a day and hour in a given region

In [1]:
import argparse
import datetime as dt
import sys

import meteomatics.api as api

In [2]:
###Credentials:
username = 'hackz_bucher_3'
password = 'GbIj6UqbTi5C'

In [3]:
# startdate_station_ts = dt.datetime.utcnow().replace(hour=0, minute=0, second=0, microsecond=0)-dt.timedelta(days=0)
# enddate_station_ts = startdate_station_ts + dt.timedelta(days=2)
# interval_station_ts = dt.timedelta(hours=1)

In [5]:
startdate_station_ts = dt.datetime(2019,4,1,0,0) # 1 April
# enddate_station_ts = dt.datetime(2019,4,10,0,0) # 26 Sept
enddate_station_ts = dt.datetime(2019,9,26,0,0) # 26 Sept
interval_station_ts = dt.timedelta(hours=1)

#### Human Thermal comfort factors

- Temperature: 2 metres above ground in Celcius 't_2m:C'
- Relative humidity: 2 metres above ground in pascals 'relative_humidity_2m:p' --> NA
- Median Radiant temperature --> NA
- Wind speed: 10 metres above ground in meter per sec 'wind_speed_10m:ms'

#### Possibility of rain
- Precipiation: In millimeters that has fallen in one hour 'precip_1h:mm'
- Cloud cover: effective Cloud cover in Octas 'effective_cloud_cover:octas' --> NA
- Thunderstorm probability: Probability of thunderstorm for the next 24 hours 'prob_tstorm_24h' --> NA

In [21]:
parameters_station_ts = ['t_2m:C', 'wind_speed_10m:ms', 'precip_1h:mm']#,'relative_humidity_2m:p','effective_cloud_cover:octas', 'prob_tstorm_24h'

In [22]:
model_station_ts = 'mix-obs' # 'mix' for prediction and 'mix-obs' for observation

In [23]:
coordinates_station_ts = [(47.532477, 7.601737973795679)] # Coordinates of Basel

In [24]:
print("station coordinates timeseries:")
try:
    df_sd_coord = api.query_station_timeseries(startdate_station_ts, enddate_station_ts, interval_station_ts,
                                               parameters_station_ts, username, password, model=model_station_ts,
                                               latlon_tuple_list=coordinates_station_ts, on_invalid='fill_with_invalid',
                                               request_type="POST", temporal_interpolation='none')
    print(df_sd_coord.head())
except Exception as e:
    print("Failed, the exception is {}".format(e))

station coordinates timeseries:
2019-09-28 04:09:20| INFO |Calling URL: https://api.meteomatics.com/2019-04-01T00:00:00+00:00--2019-09-26T00:00:00+00:00:PT1H (username = hackz_bucher_3)
                                                       t_2m:C  \
station_id                  validdate                           
47.532477,7.601737973795679 2019-04-01 00:00:00+00:00     8.0   
                            2019-04-01 01:00:00+00:00     7.0   
                            2019-04-01 02:00:00+00:00     6.3   
                            2019-04-01 03:00:00+00:00     5.1   
                            2019-04-01 04:00:00+00:00     4.8   

                                                       wind_speed_10m:ms  \
station_id                  validdate                                      
47.532477,7.601737973795679 2019-04-01 00:00:00+00:00                1.5   
                            2019-04-01 01:00:00+00:00                2.4   
                            2019-04-01 02:00:00+00:00 

/media/mayur/4849a1fc-787d-4ad7-83d7-cf186eda025b/mayur/anaconda3/envs/hack-zur-2019/lib/python3.7/site-packages/meteomatics/api.py:195: FutureWarning: from_items is deprecated. Please use DataFrame.from_dict(dict(items), ...) instead. DataFrame.from_dict(OrderedDict(items)) may be used to preserve the key order.
  df = pd.DataFrame.from_items(dict_data.items(), orient="index", columns=parameters_ts)


In [25]:
maximum_temperature = df_sd_coord['t_2m:C'].max()
# minimum_windspeed = df_sd_coord['wind_speed_10m:ms'].min()
mean_temperature = df_sd_coord['t_2m:C'].mean()
just_precipitation = df_sd_coord['precip_1h:mm']
first_value_of_precipitation = df_sd_coord.iloc[0]['precip_1h:mm']
just_at_12UTC = df_sd_coord[df_sd_coord.index.get_level_values('validdate').hour == 12]

In [26]:
df_sd_coord[]

t_2m:C  \
station_id                  validdate                           
47.532477,7.601737973795679 2019-04-01 00:00:00+00:00     8.0   
                            2019-04-01 01:00:00+00:00     7.0   
                            2019-04-01 02:00:00+00:00     6.3   
                            2019-04-01 03:00:00+00:00     5.1   
                            2019-04-01 04:00:00+00:00     4.8   

                                                       wind_speed_10m:ms  \
station_id                  validdate                                      
47.532477,7.601737973795679 2019-04-01 00:00:00+00:00                1.5   
                            2019-04-01 01:00:00+00:00                2.4   
                            2019-04-01 02:00:00+00:00                1.8   
                            2019-04-01 03:00:00+00:00                2.1   
                            2019-04-01 04:00:00+00:00                2.0   

                                                       precip_1h:mm  
station_id                  validdate                                
47.532477,7.601737973795679 2019-04-01 00:00:00+00:00           0.0  
                            2019-04-01 01:00:00+00:00           0.0  
                            2019-04-01 02:00:00+00:00           0.0  
                            2019-04-01 03:00:00+00:00           0.0  
                            2019-04-01 04:00:00+00:00           0.0

In [27]:
export_csv = df_sd_coord.to_csv (r'./weather_data.csv', index = True, header=True) #Don't forget to add '.csv' at the end of the path